In [0]:
ROOT='/content/drive/My Drive/dl/美团/CLS/'
BERT_ROOT='/content/drive/My Drive/dl/google-quest-challenge/pretrained-bert-models-for-pytorch/新建文件夹/bert-base-chinese/'
BERT_TOK_ROOT='/content/drive/My Drive/dl/google-quest-challenge/pretrained-bert-models-for-pytorch/bert-base-chinese-vocab.txt'
MAX_LEN=22

In [0]:
!pip install pytorch_pretrained_bert
!pip install transformers
!pip install scikit-learn==0.21.3

     |████████████████████████████████| 133kB 4.5MB/s 
     |████████████████████████████████| 501kB 5.1MB/s 
     |████████████████████████████████| 1.0MB 55.0MB/s 
     |████████████████████████████████| 3.7MB 44.0MB/s 
     |████████████████████████████████| 870kB 37.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=24f08f4186ab7f0a3a6bbceebc886b12358ce3839a77fe1f4f111c89f234268c
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
     |████████████████████████████████| 6.7MB 5.1MB/s 
  Found existing installation: scikit-learn 0.22.1
    Uninstalling scikit-learn-0.22.1:
      Successfully uninstalled scikit-learn-0.22.1


In [0]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [0]:
import sklearn
import torch
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from pytorch_pretrained_bert import BertModel, BertTokenizer
root='/content/drive/My Drive/google-quest-challenge/'
import numpy as np
import time
import tqdm
import math
import scipy
from tqdm import tqdm_notebook
import torch.nn.functional as F
import random
import transformers, sys, os, gc
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, BertConfig,
    WEIGHTS_NAME, CONFIG_NAME, AdamW, get_linear_schedule_with_warmup, 
    get_cosine_schedule_with_warmup,
)
from torch.utils.data import DataLoader, Dataset,RandomSampler, SequentialSampler
from transformers.modeling_bert import BertPreTrainedModel 
from sklearn.model_selection import StratifiedShuffleSplit,StratifiedKFold
from tqdm import tqdm_notebook
from sklearn.metrics import roc_curve 
import numpy as np

美团10w


In [0]:
df = pd.read_excel('/content/drive/My Drive/dl/美团/data/评论.csv')
df

,shopid,data,content,id,score
0,"144693498362135320,""",2018-10-26,#往后余生#很惊艳，女朋友很喜欢，很满意的一次购物，服务态度也好，满足了我的要求,E***4,5.0
1,"144693498362135320,""",2018-07-11,#一生所爱#真漂亮，同事们都说好看，谢谢老公送的蛋糕，天鹅是可以吃的巧克力做的，很赞，总之很开心。,C***静,5.0
2,"144693498362135320,""",2018-09-29,……味道不错……,琪***…,5.0
3,"144693498362135320,""",2018-01-28,棒棒的 水果多而且奶油不腻 很好的蛋糕 鼓励下#好好吃的浆果蛋糕#,匿***户,5.0
4,"144693498362135320,""",2017-12-06,包装赞 口味佳 很适合女孩子 发光生日帽特别有气氛,匿***户,5.0
...,...,...,...,...,...
79311,"144955603036378139,""",2017-01-18,这是我买过的最轻的蛋糕了，外观还可以就是不知道吃起来怎么样，但愿好吃吧,f***6,4.0
79312,"144955603036378139,""",2017-03-05,真是大大的惊喜，物超所值，不锈钢的精致餐具，水果丰富，品质太高了；一下成粉丝啊，哈哈，大赞！,B***2,5.0
79313,"144955603036378139,""",2017-02-13,真是在买蛋糕啊，连个小盘子 叉子蜡烛什么都没有，对的起这个价格吗 坑爹,s***5,3.0
79314,"144955603036378139,""",2018-05-27,之前在这家定过生日蛋糕觉得挺好，本着信任原则定了冰淇淋蛋糕，但这个冰淇淋蛋糕实在是恕不敢恭维...,你***哥,1.0


In [0]:
df['score'].isnull().sum()

2179

# **补全**

In [0]:
df['score']=df['score'].fillna(3)


In [0]:
labels = df['score'].values

In [0]:
import re
def clean_numbers(text):
  if type(text)!= int:
    text = text.strip().replace(' ','').replace('\t','')
    text=re.sub(r'\W+',' ',text)
  return text

In [0]:
labels_sent = []

In [0]:
df['content']=df['content'].apply(lambda x: clean_numbers(x))

In [0]:
train_data=df['content'].values
train_label_sent = torch.tensor(df['score'].values-1,dtype=torch.int).long()

In [0]:
train_meituan=[]

In [0]:
for line in range(len(train_data)):
  if type(train_data[line]) != int:
    train_data[line] = train_data[line].replace(' ','')
    if len(train_data[line])<=20:
      train_meituan.append(train_data[line])
      labels_sent.append(labels[line])

In [0]:
tokenizer=BertTokenizer.from_pretrained(BERT_TOK_ROOT)
train_meituan_idx = []
train_meituan_mask = []
train_meituan_segment = []
train_labels_meituan = []
train_meituan_tfidf = []

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [0]:
lenth_meituan=[]

In [0]:
for line in range(len(train_meituan)):
  token = tokenizer.tokenize(train_meituan[line])
  token = ["[CLS]"] + token + ["[SEP]"]
  if len(token)>MAX_LEN:
    continue
  train_meituan_tfidf.append(train_meituan[line])
  train_labels_meituan.append(labels_sent)
  lenth_meituan.append(len(token))
  if len(token)> MAX_LEN:
      print(len(token))
      raise IndexError("Token length more than max seq length!")

  mask = [1]*len(token)+ [0]*(MAX_LEN-len(token))
  sent = [0]*MAX_LEN
  idx = tokenizer.convert_tokens_to_ids(token)
  idx = idx + (MAX_LEN-len(idx))*[0]
  train_meituan_idx.append(idx)
  train_meituan_mask.append(mask)
  train_meituan_segment.append(sent)

# **饿了么**

In [0]:
train_elm_txt=[]
labels_elm=[]

In [0]:
with open('/content/drive/My Drive/dl/美团/data/waimai_10k.csv.txt','r') as f:
  for line in f:
    data=line.strip().replace(' ','').replace('\n','').replace(',','')
    data=re.sub(r'\W+',' ',data)
    labels_elm.append(data[0])
    train_elm_txt.append(data[1:])

In [0]:
train_elm_idx = []
train_elm_mask = []
train_elm_segment = []
lenth_elm = []
train_elm_tiidf = []

In [0]:
for line in range(len(train_elm_txt)):
  token = tokenizer.tokenize(train_elm_txt[line])
  token = ["[CLS]"] + token + ["[SEP]"]
  if len(token)>MAX_LEN:
    continue
  train_elm_tiidf.append(train_elm_txt[line])
  lenth_elm.append(len(token))
  if len(token)> MAX_LEN:
      print(len(token))
      raise IndexError("Token length more than max seq length!")

  mask = [1]*len(token)+ [0]*(MAX_LEN-len(token))
  sent = [0]*MAX_LEN
  idx = tokenizer.convert_tokens_to_ids(token)
  idx = idx + (MAX_LEN-len(idx))*[0]
  train_elm_idx.append(idx)
  train_elm_mask.append(mask)
  train_elm_segment.append(sent)

In [0]:
len(train_meituan_tfidf),len(train_elm_tiidf)

(48552, 7814)

In [0]:
train_tfidf = train_meituan_tfidf+train_elm_tiidf

In [0]:
len(train_tfidf)

56366

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [0]:
# text_encoder = Pipeline([
#     ('Text-TF-IDF', TfidfVectorizer(ngram_range=(1, 3))),
#     ('Text-SVD', TruncatedSVD(n_components = 600))], verbose=True)

In [0]:
# text_encoder=torch.load('/content/drive/My Drive/dl/推荐评论/text_encoder.pkl')

In [0]:
# tfidf=text_encoder.fit_transform(train_tfidf)

[Pipeline] ....... (step 1 of 2) Processing Text-TF-IDF, total=   0.4s
[Pipeline] .......... (step 2 of 2) Processing Text-SVD, total= 1.5min


In [0]:
# torch.save(train_tfidf,'/content/drive/My Drive/dl/推荐评论/CNN/w20_train_tfidf.pkl')

In [0]:
lenths = np.array(lenth_meituan+lenth_elm)
lenths=lenths.reshape(-1,1)

In [0]:
# use = np.hstack( (lenths,tfidf))
# print(use.shape)

(56366, 901)


In [0]:
# tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [0]:
# torch.save(use,train_tfidf)

In [0]:
train_w20_idx=train_meituan_idx+train_elm_idx
train_w20_mask = train_meituan_mask+train_elm_mask
train_w20_segment = train_meituan_segment+train_elm_mask
train_w20_lenths = lenth_meituan+lenth_elm

In [0]:
train_w20_lenths = np.array(train_w20_lenths)
train_w20_lenths=train_w20_lenths.reshape(-1,1)
use = train_w20_lenths

In [0]:
use_shape = use.shape[1]
use = list(use)

In [0]:

## Stolen from transformer code base without any noble intention.

## Stolen from transformer code base without any noble intention.
class Attention(nn.Module):
    def __init__(self, feature_dim, step_dim, bias=True, **kwargs):
        super(Attention, self).__init__(**kwargs)
        
        self.supports_masking = True

        self.bias = bias
        self.feature_dim = feature_dim
        self.step_dim = step_dim
        self.features_dim = 0
        
        weight = torch.zeros(feature_dim, 1)
        nn.init.xavier_uniform_(weight)
        self.weight = nn.Parameter(weight)
        
        if bias:
            self.b = nn.Parameter(torch.zeros(step_dim))
        
    def forward(self, x, mask=None):
        feature_dim = self.feature_dim
        step_dim = self.step_dim

        eij = torch.mm(
            x.contiguous().view(-1, feature_dim), 
            self.weight
        ).view(-1, step_dim)
        
        if self.bias:
            eij = eij + self.b
            
        eij = torch.tanh(eij)
        a = torch.exp(eij)
        
        if mask is not None:
            a = a * mask

        a = a / torch.sum(a, 1, keepdim=True) + 1e-10
        
        weighted_input = x.permute(1,0,2) * torch.unsqueeze(a, -1)
        return torch.sum(weighted_input, 1)
class cnnbert(BertPreTrainedModel):

    def __init__(self, config,kernel_sizes,num_channels,use_shape):
        super(cnnbert, self).__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(0.3)
        self.bn = nn.LayerNorm(768)
        self.gelu=GELU()
        self.att1=Attention(512,51)
        self.att2=Attention(512,50)
        self.att3=Attention(512,49)
        self.att4=Attention(512,48)
        self.att5=Attention(512,43)
        self.softmax=nn.Softmax(dim=1)
        self.pre_cnn=nn.Linear(1*sum(num_channels)+use_shape,1)
        self.linear = nn.Linear(1*sum(num_channels)+use_shape,1*sum(num_channels)+use_shape)

        self.ln_cnn=nn.LayerNorm(1*sum(num_channels))
        self.wt=torch.nn.Parameter(torch.rand((13,1),dtype=torch.float32,device=torch.device('cuda'),requires_grad=True))
        nn.init.xavier_normal_(self.wt)
        self.pool = GlobalAVGPool1d()
        self.convs = nn.ModuleList()
        for c, k in zip(num_channels, kernel_sizes):
          self.convs.append(nn.Conv1d(in_channels = 768,
          out_channels = c,
          kernel_size = k))

    def forward(
        self,
        use,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        
    ):

        outputs = self.bert(
           
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        w1=outputs[2] 
        batch_size1,len_seq1,e_s1=w1[1].size()
        bert_output1=w1[0].to(device)
        bert_output1=bert_output1.unsqueeze(1)
        for i in range(1,13):
          bert_output1=torch.cat((bert_output1,w1[i].to(device).unsqueeze(1)),dim=1)
        bert_output1=bert_output1.view(batch_size1,13,-1)
        bert_output1=bert_output1.permute(0,2,1)
        bert_output1=torch.matmul(bert_output1,self.softmax((self.wt)))
        bert_output1=bert_output1.view(batch_size1,len_seq1,e_s1)#下可接CNN或者att
        bert_output1=self.bn(bert_output1)
        bert_output1=bert_output1.permute(0, 2, 1)

        bert_output1 = self.dropout(bert_output1)
        '''conout1=self.att1(self.gelu(self.convs[0](bert_output1).permute(2,0,1)))
        conout2=self.att2(self.gelu(self.convs[1](bert_output1).permute(2,0,1)))
        conout3=self.att3(self.gelu(self.convs[2](bert_output1).permute(2,0,1)))
        conout4=self.att4(self.gelu(self.convs[3](bert_output1).permute(2,0,1)))
        conout5=self.att5(self.gelu(self.convs[4](bert_output1).permute(2,0,1)))
        conout=torch.cat((conout1,conout2,conout3,conout4,conout5),dim=1)'''
        encoding =torch.cat([self.pool(self.gelu(conv(bert_output1))).squeeze(-1) for conv in self.convs], dim=1)
       
        encoding=torch.cat((encoding,use),dim=1)
        
        logits = self.pre_cnn((encoding))
        return logits 

In [0]:
class GlobalAVGPool1d(nn.Module):
    def __init__(self):
        super(GlobalAVGPool1d, self).__init__()
    def forward(self, x):
         # x shape: (batch_size, channel, seq_len)
        return F.avg_pool1d(x, kernel_size=x.shape[2]) # shape: (batch_size, channel, 1)

In [0]:
class GELU(nn.Module):
    """
    Paper Section 3.4, last paragraph notice that BERT used the GELU instead of RELU
    """

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))

In [0]:
class EvalDataset(Dataset):
  def __init__(self,train_idx,train_mask,train_segment,use,label=None):
    self.train_idx = torch.tensor(train_idx,dtype=torch.int).long()
    self.train_mask = torch.tensor(train_mask,dtype=torch.int).long()
    self.train_segment = torch.tensor(train_segment,dtype=torch.int).long()
    self.use = torch.tensor(use , dtype=torch.float32)
    self.label = None
    if label is not None:
      self.label = torch.tensor(label,dtype=torch.float32)
  def __len__(self):
    return len(self.train_idx)
  def __getitem__(self,item):
    if self.label is not None:

      output = {
        'train_idx' : self.train_idx[item],
        'train_mask' : self.train_mask[item],
        'train_segment' :self.train_segment[item],
        'use':self.use[item],
        'label' : self.label[item]
    }
    else:
      output = {
        'train_idx' : self.train_idx[item],
        'train_mask' : self.train_mask[item],
        'use' : self.use[item],
        'train_segment' :self.train_segment[item],
      }
    return {key: value for key, value in output.items()}

In [0]:
bert_model_config = BERT_ROOT+'/bert_config.json'
bert_config = BertConfig.from_json_file(bert_model_config)
bert_config.num_labels = 1
bert_config.output_hidden_states=True

In [0]:
wdataset = EvalDataset(train_w20_idx,train_w20_mask,train_w20_segment,train_w20_lenths)

In [0]:
pre_shape = len(train_w20_idx)
pre_loader = DataLoader(wdataset,batch_size=128,shuffle=False)
pred = np.zeros((pre_shape))

In [0]:
device = torch.device('cuda')

In [0]:
def predict_model(model,predict_loader ,pre_shape,batch_size ,config=None):
  ema.apply_shadow()
  model.eval()
  acc_lm = 0
  acc_sent = 0
  avg_loss = 0
  pred = np.zeros((pre_shape))
  with torch.no_grad():
    for idx, batch in tqdm_notebook(enumerate(predict_loader),mininterval=2,desc='--Valing',leave=False):
      input_ids = batch['train_idx'].to(device)
      input_mask = batch['train_mask'].to(device)
      input_segments = batch['train_segment'].to(device)
      use = batch['use'].to(device)
      output_train = model(use=use,input_ids = input_ids.long(),
                              labels = None,
                              attention_mask = input_mask,
                              token_type_ids = input_segments,)
      pred[idx*batch_size : (idx+1)*batch_size]=(torch.sigmoid(output_train.cpu().view(-1)))
  ema.restore()
  return pred

In [0]:
for i in range(1,6):
  kernel_sizes,nums_channels = [2,3,4,5,10], [512,512,512,512,512]
  model = cnnbert.from_pretrained(BERT_ROOT,kernel_sizes=kernel_sizes,num_channels=nums_channels,use_shape=use_shape,config = bert_config)
  model.to(device)
  model.load_state_dict(torch.load('/content/drive/My Drive/dl/推荐评论/CNN/best_param_score_{}'.format(i)))
  pred += predict_model(model,pre_loader ,pre_shape,128, config=None)'''

In [0]:
(((pred/5)>0.5).astype(int)==1).sum()

In [0]:
torch.save(train_w20_idx,'/content/drive/My Drive/dl/推荐评论/CNN/w20_train_idx.pkl')
torch.save(train_w20_mask,'/content/drive/My Drive/dl/推荐评论/CNN/w20_train_mask.pkl')
torch.save(train_w20_segment,'/content/drive/My Drive/dl/推荐评论/CNN/w20_train_segment.pkl')
torch.save(train_w20_lenths,'/content/drive/My Drive/dl/推荐评论/CNN/w20_train_lenth.pkl')
torch.save(pred/5,'/content/drive/My Drive/dl/推荐评论/CNN/w20_train_label_notbin.pkl')
torch.save(((pred/5)>0.5).astype(int),'/content/drive/My Drive/dl/推荐评论/CNN/w20_train_label_bin.pkl')

In [0]:
(torch.load('/content/drive/My Drive/dl/推荐评论/CLS/label.pkl')==0).sum()

对等